In [ ]:
!pip install rasterio geopandas pyproj pandas numpy -q

import pandas as pd
import numpy as np
import rasterio
from pyproj import CRS, Transformer
import os
from google.colab import drive


In [ ]:
print(f"Répertoire courant : {os.getcwd()}")

try:
    drive.mount('/content/drive')
    print("--- Google Drive monté ---")
except Exception as e:
    print(f"Erreur lors du montage de Google Drive : {e}")

# Dossiers
base_data_root = '/content/drive/MyDrive/Stage_CEFREM_Salma_2025/Data'

# Chemins fichiers d'entrée
stations_metadata_input_path = os.path.join(base_data_root, 'Meteo', 'InSitu', 'Consolidated_All_Pyrenees_Stations_Metadata.csv')
mnt_30m_final_path = os.path.join(base_data_root, 'MNT', 'Pyrenees_MNT_30m_Reprojected.tif')
slope_tif_path = os.path.join(base_data_root, 'MNT', 'Pyrenees_Slope_30m_New.tif')
aspect_tif_path = os.path.join(base_data_root, 'MNT', 'Pyrenees_Aspect_30m_New.tif')
aspect_cos_tif_path = os.path.join(base_data_root, 'MNT', 'Pyrenees_Aspect_cos_New.tif')
aspect_sin_tif_path = os.path.join(base_data_root, 'MNT', 'Pyrenees_Aspect_sin_New.tif')

# Fichier de sortie
output_dir = os.path.join(base_data_root, 'Meteo', 'InSitu')
output_csv_filename = 'Consolidated_Stations_With_Raster_Values_And_Aspect_Components.csv'
output_csv_path = os.path.join(output_dir, output_csv_filename)


In [ ]:
def get_raster_value_at_coords(src_raster_obj, lat, lon, raster_name=""):
    try:
        if not isinstance(src_raster_obj, rasterio.io.DatasetReader):
            print(f"[DEBUG] {raster_name}: L'objet raster est invalide. Type: {type(src_raster_obj)}")
            return np.nan

        crs_raster_raw = src_raster_obj.crs
        try:
            crs_raster = CRS.from_wkt(crs_raster_raw.wkt)
        except Exception as crs_e:
            print(f"[DEBUG] {raster_name}: Erreur de conversion CRS : {crs_e}")
            return np.nan

        crs_wgs84 = CRS("EPSG:4326")
        if crs_raster != crs_wgs84:
            transformer = Transformer.from_crs(crs_wgs84, crs_raster, always_xy=True)
            x_raster, y_raster = transformer.transform(lon, lat)
        else:
            x_raster, y_raster = lon, lat

        row, col = src_raster_obj.index(x_raster, y_raster)

        if not (0 <= row < src_raster_obj.height and 0 <= col < src_raster_obj.width):
            return np.nan

        value = next(src_raster_obj.sample([(x_raster, y_raster)]))[0]
        if src_raster_obj.nodata is not None and value == src_raster_obj.nodata:
            return np.nan
        elif np.isclose(value, -9999.0):
            return np.nan
        else:
            return value

    except rasterio.errors.RasterioIOError as rio_e:
        print(f"RasterioIOError pour {raster_name} ({lat}, {lon}) : {rio_e}")
        return np.nan
    except Exception as e:
        print(f"Erreur inconnue pour {raster_name} ({lat}, {lon}) : {e}")
        return np.nan


In [ ]:
print("\n--- Ouverture des fichiers raster ---")

try:
    if os.path.exists(mnt_30m_final_path):
        src_mnt = rasterio.open(mnt_30m_final_path)
        print(f"MNT chargé. CRS: {src_mnt.crs}")
    else:
        raise FileNotFoundError(f"MNT manquant : {mnt_30m_final_path}")

    if os.path.exists(slope_tif_path):
        src_slope = rasterio.open(slope_tif_path)
        print(f"Pente chargée. CRS: {src_slope.crs}")
    else:
        raise FileNotFoundError(f"Pente manquante : {slope_tif_path}")

    if os.path.exists(aspect_tif_path):
        src_aspect = rasterio.open(aspect_tif_path)
        print(f"Aspect (degrés) chargé. CRS: {src_aspect.crs}")
    else:
        raise FileNotFoundError(f"Aspect manquant : {aspect_tif_path}")

    if os.path.exists(aspect_cos_tif_path):
        src_aspect_cos = rasterio.open(aspect_cos_tif_path)
        print(f"Aspect COS chargé. CRS: {src_aspect_cos.crs}")
    else:
        raise FileNotFoundError(f"Aspect COS manquant : {aspect_cos_tif_path}")

    if os.path.exists(aspect_sin_tif_path):
        src_aspect_sin = rasterio.open(aspect_sin_tif_path)
        print(f"Aspect SIN chargé. CRS: {src_aspect_sin.crs}")
    else:
        raise FileNotFoundError(f"Aspect SIN manquant : {aspect_sin_tif_path}")

except Exception as e:
    print(f" Erreur lors de l'ouverture des rasters : {e}")
    exit()


In [ ]:
print("\n--- Traitement des métadonnées de stations ---")

try:
    if os.path.exists(stations_metadata_input_path):
        df_stations_metadata = pd.read_csv(stations_metadata_input_path)
        print("Fichier des stations chargé.")
    else:
        raise FileNotFoundError(f"Fichier stations introuvable : {stations_metadata_input_path}")

    required_cols = ['LAT_station', 'LON_station']
    if not all(col in df_stations_metadata.columns for col in required_cols):
        raise ValueError(f"Colonnes manquantes : {required_cols}")

    df_stations_metadata['ALTI_MNT'] = df_stations_metadata.apply(
        lambda row: get_raster_value_at_coords(src_mnt, row['LAT_station'], row['LON_station'], "MNT"), axis=1)

    df_stations_metadata['SLOPE'] = df_stations_metadata.apply(
        lambda row: get_raster_value_at_coords(src_slope, row['LAT_station'], row['LON_station'], "Pente"), axis=1)

    df_stations_metadata['ASPECT'] = df_stations_metadata.apply(
        lambda row: get_raster_value_at_coords(src_aspect, row['LAT_station'], row['LON_station'], "Aspect"), axis=1)

    df_stations_metadata['ASPECT_COS'] = df_stations_metadata.apply(
        lambda row: get_raster_value_at_coords(src_aspect_cos, row['LAT_station'], row['LON_station'], "Aspect_COS"), axis=1)

    df_stations_metadata['ASPECT_SIN'] = df_stations_metadata.apply(
        lambda row: get_raster_value_at_coords(src_aspect_sin, row['LAT_station'], row['LON_station'], "Aspect_SIN"), axis=1)

    print("\n Extraction terminée. Exemple :")
    print(df_stations_metadata[['LAT_station', 'LON_station', 'ALTI_MNT', 'SLOPE', 'ASPECT', 'ASPECT_COS', 'ASPECT_SIN']].head())

except Exception as e:
    print(f" Erreur pendant le traitement des stations : {e}")
    exit()


In [ ]:
try:
    os.makedirs(output_dir, exist_ok=True)
    df_stations_metadata.to_csv(output_csv_path, index=False)
    print(f"\n Fichier enrichi enregistré dans : {output_csv_path}")
except Exception as e:
    print(f"Erreur lors de l'enregistrement du fichier : {e}")


In [ ]:
print("\n--- Fermeture des fichiers raster ---")

def fermer_raster(src, label):
    try:
        if src is not None and isinstance(src, rasterio.io.DatasetReader):
            src.close()
            print(f"{label} fermé.")
        else:
            print(f"{label} non fermé (non défini ou déjà fermé).")
    except Exception as e:
        print(f"Erreur en fermant {label} : {e}")

fermer_raster(src_mnt, "MNT")
fermer_raster(src_slope, "Pente")
fermer_raster(src_aspect, "Aspect")
fermer_raster(src_aspect_cos, "Aspect_COS")
fermer_raster(src_aspect_sin, "Aspect_SIN")

print("\n Fin du traitement complet.")
